**D3APL: Aplicações em Ciência de Dados** <br/>
IFSP Campinas

Renata Rabelo e Mariana Cabride

Prof. Dr. Samuel Martins (Samuka) <br/><br/>

**Descrição:** Este código utiliza a arquitetura ResNet50 com pesos pré-treinados para treinar um novo modelo. As camadas de convolução da ResNet50 são congeladas, e novas camadas densas são adicionadas no final para classificação. O otimizador Adam é usado para treinamento, e os callbacks EarlyStopping e TensorBoard são usados para parar o treinamento se o desempenho não melhorar após 10 épocas e para registrar os logs de treinamento, respectivamente.


### **Carregando as bibliotecas**

 TensorFlow, os, pandas, numpy e random são importados. Essas bibliotecas são usadas para manipulação de dados, treinamento de redes neurais e outras operações relacionadas.

In [9]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

### **Configuração dos seeds aleatórios**

Função reset_random_seeds(): Essa função configura as sementes aleatórias para garantir a reprodutibilidade dos resultados. Ela define as sementes para TensorFlow, NumPy e Python's Random.

In [11]:
def reset_random_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

reset_random_seeds()

### **Montagem do Google Drive**

O código monta o Google Drive para acessar os arquivos de dados. Ele usa a biblioteca do Google Colab para essa finalidade.

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Carregamento dos dados**

Os arquivos de treinamento e teste são carregados usando o pandas. Eles são armazenados em dataframes (dataset_df_train_completo e dataset_df_test, respectivamente). Após essa etapa, é feita a conversão dos rótulos das classes de formato de texto para números inteiros usando o LabelEncoder.

In [13]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/Kaggle/ifsp-d3apl-2023-face-recognition'
train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

dataset_df_train_completo = pd.read_csv(train_file)
dataset_df_test = pd.read_csv(test_file)

# Conversão dos rótulos em formato de texto para números inteiros
label_encoder = LabelEncoder()
dataset_df_train_completo["label"] = label_encoder.fit_transform(dataset_df_train_completo["label"])

### **Divisão do conjunto de treinamento completo em treinamento e validação**

O conjunto de treinamento completo (dataset_df_train_completo) é dividido em conjuntos de treinamento e validação usando a função train_test_split do scikit-learn. Os dados são divididos em uma proporção de 80% para treinamento e 20% para validação. A variável labels contém as classes de destino.

In [14]:
labels = dataset_df_train_completo["label"]
dataset_df_train, dataset_df_val = train_test_split(dataset_df_train_completo, test_size=0.2, random_state=42, stratify=labels)

### **Construção da arquitetura da rede neural convolucional**

In [15]:
def build_model(input_shape=(100, 100, 3), n_classes=83):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(n_classes, activation='softmax')
    ])

    return model

input_shape = (100, 100, 3)
n_classes = 83

model = build_model(input_shape, n_classes)
opt = Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dense_3 (Dense)             (None, 83)                21331     
                                                                 
Total params: 24,133,587
Trainable params: 545,875
Non-trainable params: 23,587,712
_________________________________________________________________


### **Pré-processamento dos dados**

A função preprocess_faces_dataset é definida para pré-processar os dados de imagem. Ela itera sobre o dataframe do dataset de treinamento ou validação, carrega cada imagem usando a função load_img do Keras, redimensiona para o tamanho desejado, converte para um array NumPy e normaliza os valores dos pixels dividindo por 255. O resultado é um array de imagens (X) e um array de rótulos (y).

In [16]:
def preprocess_faces_dataset(dataset_df, input_shape=(100, 100), verbose=0):
    image_list = []
    label_list = []

    for index, row in dataset_df.iterrows():
        img_path = os.path.join(data_dir, 'train', row['image-pathname'])
        img = load_img(img_path, target_size=input_shape)
        img = img_to_array(img) / 255.0
        image_list.append(img)
        label_list.append(row['label'])

        if verbose and (index % verbose) == 0:
            print(f'{index + 1}/{len(dataset_df)} - {img_path}')

    X = np.array(image_list)
    y = np.array(label_list)

    return X, y

Os conjuntos de treinamento e validação são pré-processados usando a função preprocess_faces_dataset.

In [17]:
X_train, y_train = preprocess_faces_dataset(dataset_df_train, input_shape)
X_val, y_val = preprocess_faces_dataset(dataset_df_val, input_shape)

### **Callbacks**

No código abaixo é feita a definição dos callbacks early_stopping_cb e tensorboard_callback para monitorar o treinamento e evitar overfitting.

In [18]:
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir='logs/')

## **Treinamento do modelo**

Treinamento do modelo usando o método fit, passando os dados de treinamento e validação, número de epochs, tamanho do lote e callbacks

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=50, validation_data=(X_val, y_val), callbacks=[early_stopping_cb, tensorboard_callback])

Epoch 1/100
156/156 [==============================] - 505s 3s/step - loss: 4.3490 - accuracy: 0.0267 - val_loss: 4.3394 - val_accuracy: 0.0277
Epoch 2/100
156/156 [==============================] - 506s 3s/step - loss: 4.3435 - accuracy: 0.0290 - val_loss: 4.3407 - val_accuracy: 0.0323
Epoch 3/100
156/156 [==============================] - 507s 3s/step - loss: 4.3397 - accuracy: 0.0280 - val_loss: 4.3316 - val_accuracy: 0.0277
Epoch 4/100
156/156 [==============================] - 507s 3s/step - loss: 4.3339 - accuracy: 0.0290 - val_loss: 4.3300 - val_accuracy: 0.0318
Epoch 5/100
156/156 [==============================] - ETA: 0s - loss: 4.3290 - accuracy: 0.0304

#### **Visualizando o histórico de treinamento**

O código abaixo cria um DataFrame do pandas chamado history_df a partir do objeto history.history, que contém as métricas de treinamento e validação ao longo das épocas durante o treinamento do modelo.

Em seguida, são plotados dois gráficos. O primeiro gráfico mostra as curvas de perda (loss) de treinamento e validação ao longo das épocas. O eixo x representa as épocas e o eixo y representa a perda. O segundo gráfico mostra as curvas de acurácia (accuracy) de treinamento e validação ao longo das épocas. O eixo x representa as épocas e o eixo y representa a acurácia.

Esses gráficos ajudam a visualizar o desempenho do modelo durante o treinamento e a analisar a convergência e o overfitting. A função plot do pandas é usada para traçar os gráficos e as funções grid, xlabel e ylabel do matplotlib.pyplot são usadas para adicionar a grade e rótulos aos eixos dos gráficos.

In [ ]:
history_df = pd.DataFrame(history.history)

history_df[['loss', 'val_loss']].plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Score')

history_df[['accuracy', 'val_accuracy']].plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Score')

### **Salvando o modelo treinado**

O modelo treinado está sendo salvo em um arquivo HDF5

In [ ]:
model.save('modelResNet.h5')

### **Criação do diretório de saída para salvar os conjuntos de dados pré-processados**

O código abaixo faz a criação de um diretório de saída, e na sequencia, o salvamento dos conjuntos de dados e arquivos numpy pré-processados

In [ ]:
out_dir = '/content/drive/MyDrive/Colab Notebooks/Kaggle/preprocessed_03'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

dataset_df_train_completo.to_csv(os.path.join(out_dir, 'full_train.csv'), index=False)
dataset_df_train.to_csv(os.path.join(out_dir, 'train.csv'), index=False)
dataset_df_val.to_csv(os.path.join(out_dir, 'validation.csv'), index=False)

np.save(os.path.join(out_dir, 'X_train.npy'), X_train)
np.save(os.path.join(out_dir, 'y_train.npy'), y_train)
np.save(os.path.join(out_dir, 'X_val.npy'), X_val)
np.save(os.path.join(out_dir, 'y_val.npy'), y_val)

### **Montando o arquivo de submissão**



In [ ]:
# Define uma função chamada read_test para ler e pré-processar as imagens de teste
def read_test(dataset_df, input_shape=(100, 100), verbose=0):
    image_list = []

    for index, row in dataset_df.iterrows():
        img_path = os.path.join(data_dir, 'test', row['image-pathname'])
        img = load_img(img_path, target_size=input_shape)
        img = img_to_array(img) / 255.0
        image_list.append(img)

        if verbose and (index % verbose) == 0:
            print(f'{index + 1}/{len(dataset_df)} - {img_path}')

    X = np.array(image_list)

    return X

In [ ]:
# Chama a função read_test para carregar e pré-processar as imagens de teste
X_test = read_test(dataset_df_test, input_shape=(100, 100), verbose=1)

In [ ]:
# Faz previsões usando o modelo treinado para as imagens de teste
y_test_proba = model.predict(X_test)
y_test_pred = np.argmax(y_test_proba, axis=1)

# Converte as previsões em rótulos de classe usando label_encoder.inverse_transform
# Cria um DataFrame com as colunas "image-id" e "prediction" e o salva como arquivo CSV
df = pd.DataFrame({"image-id": dataset_df_test["image-id"], "prediction": list(label_encoder.inverse_transform(y_test_pred))})
df.to_csv("results_cnn.csv", index=False)

In [ ]:
# Repete o processo anterior, salvando o DataFrame em um arquivo chamado "results.csv"
df = pd.DataFrame({"image-id": dataset_df_test["image-id"], "prediction": list(label_encoder.inverse_transform(y_test_pred))})
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/results_ResNet.csv', index=False)